In [1]:
import xarray as xr
import matplotlib.pyplot as plt

import numpy as np

import copy
import time
from scipy import integrate

In [2]:
#Constants
a=6371220 #radius of earth in m
L=2.5E6 #Latent heat in atmosphere
L_ice=3.34e5 #Latent heat of fusion
g=9.81 #Acceleration from gravity
conv_pw=1e15 #Conversion from watts to PW
cp=1007  

In [3]:
lats = np.linspace(-90, 90, 361)
lons = np.linspace(-180, 179.375, 576)
#levels = [1000, 975, 950, 925, 900, 875, 850, 825, 800, 775, 750, 725, 700, 650, 600, 550, 500, 450, 400, 350, 300,
          #250, 200, 150, 100, 70, 50, 40, 30, 20, 10, 7, 5, 4, 3, 2, 1, .7, .5, .4, .3, .1]

#levels = [1000, 925, 850, 700, 600, 500, 400, 300, 250, 200, 150, 100, 70, 50, 30, 20, 10, 5, 1]

#levels = [1000, 925, 850, 700, 600, 500, 400, 300, 250, 200,
          #150, 100, 70, 50, 40, 30, 20, 10, 7, 5, 4, 3, 2, 1, .7, .5, .4, .3, .1]

levels = [1000, 975, 950, 925, 900, 875, 850, 825, 800, 775, 750, 725, 700, 650, 600, 550, 500,
          450, 400, 350, 300, 250, 200, 150, 100, 70, 50, 30, 20, 10, 5, 1]

geometry = 2 * np.pi * np.cos(np.deg2rad(np.asarray(lats))) * a / g

### WGET command to download

wget --content-disposition --load-cookies ~/.urs_cookies --save-cookies ~/.urs_cookies --keep-session-cookies --content-disposition -i merra_links_list.txt 

In [4]:
merra_data = xr.open_mfdataset('raw_data/MERRA2_*')

#Let's only select through 2021, the last full year
merra_data = merra_data.sel(time=slice('1980-01-01', '2021-12-31'))

merra_data

<xarray.Dataset>
Dimensions:   (time: 504, lat: 361, lon: 576, lev: 42)
Coordinates:
  * lat       (lat) float64 -90.0 -89.5 -89.0 -88.5 ... 88.5 89.0 89.5 90.0
  * lev       (lev) float64 1e+03 975.0 950.0 925.0 900.0 ... 0.5 0.4 0.3 0.1
  * lon       (lon) float64 -180.0 -179.4 -178.8 -178.1 ... 178.1 178.8 179.4
  * time      (time) datetime64[ns] 1980-01-01 1980-02-01 ... 2021-12-01
Data variables:
    PS        (time, lat, lon) float32 dask.array<chunksize=(1, 361, 576), meta=np.ndarray>
    Cov_V_T   (time, lev, lat, lon) float32 dask.array<chunksize=(1, 42, 361, 576), meta=np.ndarray>
    V         (time, lev, lat, lon) float32 dask.array<chunksize=(1, 42, 361, 576), meta=np.ndarray>
    T         (time, lev, lat, lon) float32 dask.array<chunksize=(1, 42, 361, 576), meta=np.ndarray>
    Cov_V_H   (time, lev, lat, lon) float32 dask.array<chunksize=(1, 42, 361, 576), meta=np.ndarray>
    QV        (time, lev, lat, lon) float32 dask.array<chunksize=(1, 42, 361, 576), meta=np.ndarray>
    Cov_V_QV  (time, lev, lat, lon) float32 dask.array<chunksize=(1, 42, 361, 576), meta=np.ndarray>
    H         (time, lev, lat, lon) float32 dask.array<chunksize=(1, 42, 361, 576), meta=np.ndarray>
Attributes: (12/32)
    History:                           Original file generated: Thu May  7 22...
    Filename:                          MERRA2_100.instM_3d_asm_Np.198001.nc4
    Comment:                           GMAO filename: d5124_m2_jan79.inst3_3d...
    Conventions:                       CF-1
    Institution:                       NASA Global Modeling and Assimilation ...
    References:                        http://gmao.gsfc.nasa.gov
    ...                                ...
    Contact:                           http://gmao.gsfc.nasa.gov
    identifier_product_doi:            10.5067/2E096JV59PK7
    RangeBeginningTime:                00:00:00.000000
    RangeEndingTime:                   21:00:00.000000
    DODS_EXTRA.Unlimited_Dimension:    time
    history:                           2022-09-29 15:28:15 GMT Hyrax-1.16.3 h...

In [5]:
#First get the weights

levels_array = np.array(levels[::-1]) #NOTE: WE flip the level array here
mlev = (levels_array[:-1] + levels_array[1:]) / 2
new_mlev = np.append(np.insert(mlev, 0, 0), 1000)
#Now we have the generic edges of our level boxes

WEIGHT = np.empty((len(levels), len(lats), len(lons)))
ps = merra_data.PS.mean('time').values / 100 #To get in hPa

for i in range(len(lats)):
    for j in range(len(lons)):
        
        #This looks and first makes sure that the level at each point is not below the surface pressure
        #If a point is below ths surface pressure, it gets converted to the surface pressure
        #Then we subtract out the original neighboring levels
        #This will result in some level boxes getting negative weight (where surface pressure was used)
        #So, we change those values to 0
        #Then we deal with converting hPa to Pa and making units work with g
        WEIGHT[:,i,j] = np.maximum(np.minimum(new_mlev[1:], ps[i,j] * np.ones(len(levels))) - new_mlev[:-1],
                                   np.zeros(len(levels))) * 100 / g

#This is our generic weighting, used later
weight = (new_mlev[:-1] - new_mlev[1:]) * 100 / g

In [6]:
#This is basically just making a topography maks

tmask = np.ones((len(levels), len(lats), len(lons)))
TMASK = np.ones((len(levels), len(lats), len(lons)))

for i in range(len(levels)):
    for j in range(len(lats)):
        lon_idxs = np.where(ps[j,:] < new_mlev[i+1])

        tmask[i,j,lon_idxs] = 0
        TMASK[i,j,lon_idxs] = np.nan

In [7]:
#This is the weighting for each point, allowing for us to take zonal averages more easily

ZON_NORM = np.ones((len(levels), len(lats), len(lons)))

for i in range(len(levels)):
    for j in range(len(lats)):
        ZON_NORM[i,j,:] = WEIGHT[i,j,:] / np.maximum(np.sum(WEIGHT[i,j,:]), 1e-5)

In [8]:
#And zonal weights, useful for AHT where we have zonal-mean quantities

ZON_WEIGHT = np.ones((len(levels), len(lats)))

for i in range(len(levels)):
    ZON_WEIGHT[i,:] = np.mean(WEIGHT[i,:,:], axis=1)

### Now, let's save out all the weighting info to a netcdf

In [9]:
WEIGHT_attrs = 'Weighting for each grid cell. Not normalized.'
weight_attrs = 'Generic weighting for the column. Does not account for surface pressure.'
TMASK_attrs = 'Simple mask where points are 1 if above time-mean surface pressure, or nan if below. Unitless.'
tmask_attrs = 'Simple mask where points are 1 if above time-mean surface pressure, or 0 if below. Unitless.'
ZON_NORM_attrs = 'Basically a normalized version of WEIGHT, where the normalization is over longitude. To take a zonal average, multiply a quantity by this and sum along the longitude dimenion. Unitless.'
ZON_WEIGHT_attrs = 'Zonal average weight.'

weighting_ds = xr.Dataset(
                    data_vars = dict(
                        WEIGHT=(['level','latitude','longitude'], WEIGHT, {'Description':WEIGHT_attrs, 'units':'kg/m^2'}),
                        weight=(['level'], weight, {'Description':weight_attrs, 'units':'kg/m^2'}),
                        TMASK=(['level','latitude','longitude'], TMASK, {'Description':TMASK_attrs, 'units':'Unitless'}),
                        tmask=(['level','latitude','longitude'], tmask, {'Description':tmask_attrs, 'units':'Unitless'}),
                        ZON_NORM=(['level','latitude','longitude'], ZON_NORM, {'Description':ZON_NORM_attrs, 'units':'Unitless'}),
                        ZON_WEIGHT=(['level','latitude'], ZON_WEIGHT, {'Description':ZON_WEIGHT_attrs, 'units':'kg/m^2'}),
                    ),
                    coords=dict(
                        level=levels[::-1],
                        latitude=lats,
                        longitude=lons)
)

#weighting_ds.to_netcdf('merra_aht_weights.nc')

### Get MERRA data on the 19 pressure levels

In [10]:
merra_data = merra_data.interp(lev = levels)

/tdat/tylersc/mini3/envs/jupyter_copy/lib/python3.9/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/tdat/tylersc/mini3/envs/jupyter_copy/lib/python3.9/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array

### Extrapolate MERRA data so it's more like other reanalysis

In [11]:
def extrap_nans(da, extrap_method):
    levels = da.lev.values

    nan_idxs = np.isnan(da)
    
    no_nans = da.dropna('lev')
    
    interped = no_nans.interp(lev=levels,method=extrap_method,kwargs={"fill_value": "extrapolate"})
    
    new_data = xr.where(nan_idxs, interped, da)
    #new_da = da.copy(data=new_data)
    
    return new_data

In [18]:
vcomp_zonal = merra_data.V.mean(['time', 'lon']).compute()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
vcomp_zonal.plot.contourf(levels=np.linspace(-1, 1, 30))
#height_mean.plot.contourf(levels=np.linspace(0, 100000, 30))

plt.ylim([0, 50])
#plt.ylim([900, 1000])   
plt.gca().invert_yaxis()

plt.show()

In [12]:
extrap_method='linear'
merra_data['V'] = extrap_nans(merra_data.V, extrap_method)
merra_data['QV'] = extrap_nans(merra_data.QV, extrap_method)
merra_data['T'] = extrap_nans(merra_data.T, extrap_method)
merra_data['H'] = extrap_nans(merra_data.H, extrap_method)

merra_data['Cov_V_T'] = extrap_nans(merra_data.Cov_V_T, extrap_method)
merra_data['Cov_V_QV'] = extrap_nans(merra_data.Cov_V_QV, extrap_method)
merra_data['Cov_V_H'] = extrap_nans(merra_data.Cov_V_H, extrap_method)


/tdat/tylersc/mini3/envs/jupyter_copy/lib/python3.9/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/tdat/tylersc/mini3/envs/jupyter_copy/lib/python3.9/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array

### Now calculate AHT

In [13]:
def aht_merra(time_idx, correct_mass=True):
    
    #For consistency, we will extrapolate all datasets beneath the surface
    #Can specify different interpolation/extrapolation methods if desired
    #It shouldn't really matter which method is chosen
    #Load in the data and flip the level dimension, since we did this for the weighting too
    extrap_method='linear'
    v = merra_data.V.isel(time=time_idx).values[::-1,:,:]
    t = merra_data.T.isel(time=time_idx).values[::-1,:,:]
    q = merra_data.QV.isel(time=time_idx).values[::-1,:,:]
    z = merra_data.H.isel(time=time_idx).values[::-1,:,:]
    
    vt = merra_data.Cov_V_T.isel(time=time_idx).values[::-1,:,:]
    vq = merra_data.Cov_V_QV.isel(time=time_idx).values[::-1,:,:]
    vz = merra_data.Cov_V_H.isel(time=time_idx).values[::-1,:,:]


    #Now get rid of nans everywhere
    WEIGHT[np.isnan(v)] = 0
    WEIGHT[np.isnan(t)] = 0
    WEIGHT[np.isnan(q)] = 0
    WEIGHT[np.isnan(z)] = 0

    v[np.isnan(v)] = 0
    t[np.isnan(t)] = 0
    q[np.isnan(q)] = 0
    z[np.isnan(z)] = 0
    vt[np.isnan(vt)] = 0
    vq[np.isnan(vq)] = 0
    vz[np.isnan(vz)] = 0

    #Convert everything to the same MSE units
    SE = cp * t
    PE = g * z
    ME = L * q
    MSE = SE + PE + ME
    
    #Find zonal-averages
    SE_bar = np.sum(ZON_NORM * SE, axis=2)
    ME_bar = np.sum(ZON_NORM * ME, axis=2)
    PE_bar = np.sum(ZON_NORM * PE, axis=2)
    V_bar = np.sum(ZON_NORM * v, axis=2)
    E_bar = np.sum(ZON_NORM * MSE, axis=2)

    #Then find zonal anomalies
    Ve = v - V_bar[:,:,None]
    SEe = SE - SE_bar[:,:,None]
    MEe = ME - ME_bar[:,:,None]
    PEe = PE - PE_bar[:,:,None]
    
    #Multiply zonal-anomalies to get the SE piece
    #_map is just referring to plan view, ie before zonal averaging occurs
    #Dividing the WEIGHT by weight, normalizes things
    VSEse_map = (Ve * SEe) * (WEIGHT / weight[:,None,None])
    VMEse_map = (Ve * MEe) * (WEIGHT / weight[:,None,None])
    VPEse_map = (Ve * PEe) * (WEIGHT / weight[:,None,None])
    
    #Get the SEs
    VSEse_bar = np.mean(VSEse_map, axis=2)
    VMEse_bar = np.mean(VMEse_map, axis=2)
    VPEse_bar = np.mean(VPEse_map, axis=2)
    
    #Now do the TEs
    #Magically, MERRA2 outputs vt, vq, and vz, just the way we want them (with means removed)
    #So, follow same formula as SEs     
    VSEte_map = (vt * cp) * (WEIGHT / weight[:,None,None])
    VMEte_map = (vq * L) * (WEIGHT / weight[:,None,None])
    VPEte_map = (vz * g) * (WEIGHT / weight[:,None,None])

    VSEte_bar = np.mean(VSEte_map, axis=2)
    VMEte_bar = np.mean(VMEte_map, axis=2)
    VPEte_bar = np.mean(VPEte_map, axis=2)
        
    #Account for non-zero mass transport in the MMC by removing the vertically averaged v and MSE
    V_vert_ave = np.sum(V_bar * (ZON_WEIGHT / np.sum(ZON_WEIGHT, axis=0)[None,:]), axis=0)
    E_vert_ave = np.sum(E_bar * (ZON_WEIGHT / np.sum(ZON_WEIGHT, axis=0)[None,:]), axis=0)
    ME_vert_ave = np.sum(ME_bar * (ZON_WEIGHT / np.sum(ZON_WEIGHT, axis=0)[None,:]), axis=0)
    PE_vert_ave = np.sum(PE_bar * (ZON_WEIGHT / np.sum(ZON_WEIGHT, axis=0)[None,:]), axis=0)
    SE_vert_ave = np.sum(SE_bar * (ZON_WEIGHT / np.sum(ZON_WEIGHT, axis=0)[None,:]), axis=0)
    
    if correct_mass == True:
        V_bar_new = V_bar - V_vert_ave[None,:]
        E_bar_new = E_bar - E_vert_ave[None,:]
        ME_bar_new = ME_bar - ME_vert_ave[None,:]
        PE_bar_new = PE_bar - PE_vert_ave[None,:]
        SE_bar_new = SE_bar - SE_vert_ave[None,:]
    elif correct_mass == False:
        V_bar_new = V_bar
        E_bar_new = E_bar
        ME_bar_new = ME_bar
        PE_bar_new = PE_bar
        SE_bar_new = SE_bar
    else:
        print('Error with specifying mass correction choice')
        
    #MASS = np.sum(ZON_WEIGHT * V_bar * E_vert_ave[None,:], axis=0)
    MASS = np.sum(ZON_WEIGHT, axis=0) * V_vert_ave * E_vert_ave

    #Now multiply things by the needed weights and sum in the vertical
    MMC = np.sum(ZON_WEIGHT * V_bar_new * E_bar_new, axis=0)
    strm_fnct = np.cumsum((ZON_WEIGHT * V_bar_new), axis=0)[::-1,:]
    MMC_moist = np.sum(ZON_WEIGHT * V_bar_new * ME_bar_new, axis=0)
    MMC_dry = np.sum(ZON_WEIGHT * V_bar_new * SE_bar_new, axis=0)
    MMC_pot = np.sum(ZON_WEIGHT * V_bar_new * PE_bar_new, axis=0)
    
    #MASS_new = np.sum(ZON_WEIGHT * V_bar_new * E_vert_ave[None,:], axis=0)
    
    #SEs and TEs have already accounted for surface orography, so we just use generic weight here
    SE_dry = np.sum(weight[:,None] * VSEse_bar, axis=0)
    SE_pot = np.sum(weight[:,None] * VPEse_bar, axis=0)
    SE_moist = np.sum(weight[:,None] * VMEse_bar, axis=0)
    
    SE = SE_dry + SE_pot + SE_moist
    
    TE_dry = np.sum(weight[:,None] * VSEte_bar, axis=0)
    TE_pot = np.sum(weight[:,None] * VPEte_bar, axis=0)
    TE_moist = np.sum(weight[:,None] * VMEte_bar, axis=0)
    
    TE = TE_dry + TE_pot + TE_moist
    
    return MMC, MMC_moist, MMC_dry, MMC_pot, SE, SE_moist, SE_dry, SE_pot, TE, TE_moist, TE_dry, TE_pot, MASS, strm_fnct, VSEse_bar, VPEse_bar, VMEse_bar, VSEte_bar, VPEte_bar, VMEte_bar, E_bar_new, V_bar_new, SE_bar_new, PE_bar_new, ME_bar_new



In [ ]:
num_months = len(merra_data.time)

TEs = np.zeros((num_months, len(lats)))
TEs_moist = np.zeros((num_months, len(lats)))
TEs_dry = np.zeros((num_months, len(lats)))
TEs_pot = np.zeros((num_months, len(lats)))
SEs = np.zeros((num_months, len(lats)))
SEs_moist = np.zeros((num_months, len(lats)))
SEs_dry = np.zeros((num_months, len(lats)))
SEs_pot = np.zeros((num_months, len(lats)))
MMCs = np.zeros((num_months, len(lats)))
MMCs_moist = np.zeros((num_months, len(lats)))
MMCs_dry = np.zeros((num_months, len(lats)))
MMCs_pot = np.zeros((num_months, len(lats)))
MASS_fluxes = np.zeros((num_months, len(lats)))
strm_fncts = np.zeros((num_months, len(levels), len(lats)))
VSEses = np.zeros((num_months, len(levels), len(lats)))
VPEses = np.zeros((num_months, len(levels), len(lats)))
VMEses = np.zeros((num_months, len(levels), len(lats)))
VSEtes = np.zeros((num_months, len(levels), len(lats)))
VPEtes = np.zeros((num_months, len(levels), len(lats)))
VMEtes = np.zeros((num_months, len(levels), len(lats)))
V_bars = np.zeros((num_months, len(levels), len(lats)))
E_bars = np.zeros((num_months, len(levels), len(lats)))
SE_bars = np.zeros((num_months, len(levels), len(lats)))
PE_bars = np.zeros((num_months, len(levels), len(lats)))
ME_bars = np.zeros((num_months, len(levels), len(lats)))

for i in range(num_months):
    values = aht_merra(i)
    MMCs[i,:] = values[0]
    MMCs_moist[i,:] = values[1]
    MMCs_dry[i,:] = values[2]
    MMCs_pot[i,:] = values[3]
    
    SEs[i,:] = values[4]
    SEs_moist[i,:] = values[5]
    SEs_dry[i,:] = values[6]
    SEs_pot[i,:] = values[7]
    
    TEs[i,:] = values[8]
    TEs_moist[i,:] = values[9]
    TEs_dry[i,:] = values[10]
    TEs_pot[i,:] = values[11]
    
    MASS_fluxes[i,:] = values[12]
    
    strm_fncts[i,:,:] = values[13]
    
    VSEses[i,:,:] = values[14]
    VPEses[i,:,:] = values[15]
    VMEses[i,:,:] = values[16]
    
    VSEtes[i,:,:] = values[17]
    VPEtes[i,:,:] = values[18]
    VMEtes[i,:,:] = values[19]
    
    V_bars[i,:,:] = values[20]
    E_bars[i,:,:] = values[21]
    SE_bars[i,:,:] = values[22]
    PE_bars[i,:,:] = values[23]
    ME_bars[i,:,:] = values[24]

In [ ]:
TOTs = MMCs + SEs + TEs
TOTs_moist = MMCs_moist + SEs_moist + TEs_moist
TOTs_dry = MMCs_dry + SEs_dry + TEs_dry
TOTs_pot = MMCs_pot + SEs_pot + TEs_pot
TOTs_dry_pot = TOTs_dry + TOTs_pot

TEs_dry_pot = TEs_dry + TEs_pot
SEs_dry_pot = SEs_dry + SEs_pot
MMCs_dry_pot = MMCs_dry + MMCs_pot

EDDYs = SEs + TEs
EDDYs_moist = SEs_moist + TEs_moist
EDDYs_dry = SEs_dry + TEs_dry
EDDYs_pot = SEs_pot + TEs_pot
EDDYs_dry_pot = EDDYs_dry + EDDYs_pot

In [ ]:
vert_conversion =  -1000 / g

final_ds = xr.Dataset(
                    data_vars = dict(
                        total_aht=(['time','latitude'], TOTs * (geometry * g)[None,:]),
                        total_aht_dry=(['time','latitude'], TOTs_dry * (geometry * g)[None,:]),
                        total_aht_moist=(['time','latitude'], TOTs_moist * (geometry * g)[None,:]),
                        total_aht_pot=(['time','latitude'], TOTs_pot * (geometry * g)[None,:]),
                        total_aht_dry_pot=(['time','latitude'], TOTs_dry_pot * (geometry * g)[None,:]),
                        mmc_aht=(['time','latitude'], MMCs * (geometry * g)[None,:]),
                        mmc_aht_dry=(['time','latitude'], MMCs_dry * (geometry * g)[None,:]),
                        mmc_aht_moist=(['time','latitude'], MMCs_moist * (geometry * g)[None,:]),
                        mmc_aht_pot=(['time','latitude'], MMCs_pot * (geometry * g)[None,:]),
                        mmc_aht_dry_pot=(['time','latitude'], MMCs_dry_pot * (geometry * g)[None,:]),
                        se_aht=(['time','latitude'], SEs * (geometry * g)[None,:]),
                        se_aht_dry=(['time','latitude'], SEs_dry * (geometry * g)[None,:]),
                        se_aht_moist=(['time','latitude'], SEs_moist * (geometry * g)[None,:]),
                        se_aht_pot=(['time','latitude'], SEs_pot * (geometry * g)[None,:]),
                        se_aht_dry_pot=(['time','latitude'], SEs_dry_pot * (geometry * g)[None,:]),
                        te_aht=(['time','latitude'], TEs * (geometry * g)[None,:]),
                        te_aht_dry=(['time','latitude'], TEs_dry * (geometry * g)[None,:]),
                        te_aht_moist=(['time','latitude'], TEs_moist * (geometry * g)[None,:]),
                        te_aht_pot=(['time','latitude'], TEs_pot * (geometry * g)[None,:]),
                        te_aht_dry_pot=(['time','latitude'], TEs_dry_pot * (geometry * g)[None,:]),
                        eddy_aht=(['time','latitude'], EDDYs * (geometry * g)[None,:]),
                        eddy_aht_dry=(['time','latitude'], EDDYs_dry * (geometry * g)[None,:]),
                        eddy_aht_moist=(['time','latitude'], EDDYs_moist * (geometry * g)[None,:]),
                        eddy_aht_pot=(['time','latitude'], EDDYs_pot * (geometry * g)[None,:]),
                        eddy_aht_dry_pot=(['time','latitude'], EDDYs_dry_pot * (geometry * g)[None,:]),
                        mass=(['time','latitude'], MASS_fluxes * (geometry * g)[None,:]),
                        #mse_surf=(['time','latitude'], merra_data_mse_surf),
                        strm_fnct=(['time','level','latitude'], strm_fncts * g * geometry[None,None,:]),
                        VSEse=(['time','level','latitude'], VSEses * -1000 * g**2 * geometry[None,None,:]),
                        VPEse=(['time','level','latitude'], VPEses * -1000 * g**2 * geometry[None,None,:]),
                        VMEse=(['time','level','latitude'], VMEses * -1000 * g**2 * geometry[None,None,:]),
                        VSEte=(['time','level','latitude'], VSEtes * -1000 * g**2 * geometry[None,None,:]),
                        VPEte=(['time','level','latitude'], VPEtes * -1000 * g**2 * geometry[None,None,:]),
                        VMEte=(['time','level','latitude'], VMEtes * -1000 * g**2 * geometry[None,None,:]),
                        V_bar=(['time','level','latitude'], V_bars),
                        E_bar=(['time','level','latitude'], E_bars),
                        ME_bar=(['time','level','latitude'], ME_bars),
                        PE_bar=(['time','level','latitude'], PE_bars),
                        SE_bar=(['time','level','latitude'], SE_bars),
                    ),
                    coords=dict(
                        time=merra_data.time,
                        level=levels,
                        latitude=lats,)
)

final_ds.to_netcdf('merra_aht_19_levels_strato_tropo_full.nc')